In [1]:
import pandas as pd
import json
import requests
import datetime as dt
from datetime import date, timedelta, datetime
from tqdm.notebook import tqdm
import time

# Function to make the start dates list 
def days(start, end, step=timedelta(days=90)):
    curr = start
    while curr <= end:
        yield curr
        curr += step

In [83]:
# Create the date list
starting_date=date(2020, 1, 1)
date_list=list(days(starting_date, dt.date.today()))

In [84]:
# POSTtraffic snapshot Telraam to get a list of segment ids
data_segment=pd.DataFrame()
data_list2=date_list.copy()
data_list2.append(dt.date.today())
for start_date in tqdm(data_list2):
  time.sleep(2)
  url = "https://telraam-api.net/v1/reports/traffic_snapshot"
  payload = "{\r\n    \"time\":\"" +str(start_date)+ " 00:00:00Z\",\r\n    \"contents\":\"full\",\r\n    \"area\":\"7.8359048,47.9963918,10\"\r\n}\r\n"
  headers = {'X-Api-Key': '9bDufSS8AE2NAOT2R5YtR9vutyukJlZh6ZQyhTnQ'}
  response = requests.request("POST", url, headers=headers, data=payload,)
  response_json=response.json()
  response_json=pd.json_normalize(response_json['features'])  
  data_segment=pd.concat([data_segment,response_json])

segment_ids_list=data_segment['properties.segment_id'].unique().tolist()
geometry_data=data_segment[['properties.segment_id','geometry.coordinates']].drop_duplicates(subset='properties.segment_id')

  0%|          | 0/12 [00:00<?, ?it/s]

In [86]:
# POSTtraffic Telraam to get historical data
data_repot=pd.DataFrame()
for segment_id in tqdm(segment_ids_list):
    for start_date in tqdm(date_list):
        time.sleep(2)
        end_date=start_date+ dt.timedelta(days=89)
        if end_date <= dt.date.today():
            payload = "{\r\n  \"level\": \"segments\",\r\n  \"format\": \"per-hour\",\r\n  \"id\": \""+str(segment_id)+"\",\r\n  \"time_start\": \"" +str(start_date)+ " 00:00:00Z\",\r\n  \"time_end\": \""+ str(end_date)+ " 23:00:00Z\"\r\n}"
            url = "https://telraam-api.net/v1/reports/traffic"
            headers = {'X-Api-Key': '9bDufSS8AE2NAOT2R5YtR9vutyukJlZh6ZQyhTnQ'}
            response = requests.request("POST", url, headers=headers, data=payload)
            response_json=response.json()

            for result in response_json['report']:
                data=pd.DataFrame([result])
                data_repot=pd.concat([data_repot,data])

    start_date = end_date+dt.timedelta(days=-89)            
    end_date = dt.date.today()
    payload = "{\r\n  \"level\": \"segments\",\r\n  \"format\": \"per-hour\",\r\n  \"id\": \""+str(segment_id)+"\",\r\n  \"time_start\": \"" +str(start_date)+ " 00:00:00Z\",\r\n  \"time_end\": \""+ str(end_date)+ " 23:00:00Z\"\r\n}"
    url = "https://telraam-api.net/v1/reports/traffic"
    headers = {'X-Api-Key': '9bDufSS8AE2NAOT2R5YtR9vutyukJlZh6ZQyhTnQ'}
    response = requests.request("POST", url, headers=headers, data=payload)
    response_json=response.json()

    for result in response_json['report']:
        data=pd.DataFrame([result])
        data_repot=pd.concat([data_repot,data])

# Add the type of day
data_repot['weekday']=data_repot['date'].str.split('T').str[0]
data_repot['weekday'] =  pd.to_datetime(data_repot['weekday'], format='%Y-%m-%d')
data_repot['weekday'] = data_repot['weekday'].dt.weekday.apply(lambda x: 'Weekday' if x < 5 else 'Weekend')   

# Bring the location data
data_repot=pd.merge(left=data_repot,right=geometry_data,left_on='segment_id', right_on='properties.segment_id')
data_repot.drop('properties.segment_id',inplace=True, axis=1)

data_repot.to_csv(r'C:\Users\Ibrahem.Alsalamh\OneDrive - Arup\Freiburg smart city\traffic data\telraam_data_freiburg_complete.csv')


  0%|          | 0/11 [00:00<?, ?it/s]

  0%|          | 0/11 [00:00<?, ?it/s]

  0%|          | 0/11 [00:00<?, ?it/s]

  0%|          | 0/11 [00:00<?, ?it/s]

  0%|          | 0/11 [00:00<?, ?it/s]

...

  0%|          | 0/11 [00:00<?, ?it/s]

  0%|          | 0/11 [00:00<?, ?it/s]

  0%|          | 0/11 [00:00<?, ?it/s]

  0%|          | 0/11 [00:00<?, ?it/s]

  0%|          | 0/11 [00:00<?, ?it/s]

In [ ]:
# we need to understand why this is the last time 2022-07-20T20:00:00.000Z

In [ ]:
# POSTtraffic Telraam to get historical data 
url = "https://telraam-api.net/v1/reports/traffic"
payload = "{\r\n  \"level\": \"segments\",\r\n  \"format\": \"per-hour\",\r\n  \"id\": \"9000001395\",\r\n  \"time_start\": \"2022-03-30 07:00:00Z\",\r\n  \"time_end\": \"2022-06-30 09:00:00Z\"\r\n}"
headers = {'X-Api-Key': '9bDufSS8AE2NAOT2R5YtR9vutyukJlZh6ZQyhTnQ'}

response = requests.request("POST", url, headers=headers, data=payload)
response_json=response.json()

#data_repot=pd.DataFrame()
#for result in response_json['report']:
#    data=pd.DataFrame([result])
#    data_repot=pd.concat([data_repot,data])

#data_repot
#data_repot.to_csv(r'C:\Users\Ibrahem.Alsalamh\OneDrive - Arup\Freiburg smart city\traffic data\telraam_data_postTraffic.csv')

In [ ]:
url = "https://telraam-api.net/v1/reports/traffic_snapshot"
payload = "{\r\n    \"time\":\"live\",\r\n    \"contents\":\"full\",\r\n    \"area\":\"7.8359048,47.9963918,10\"\r\n}\r\n"
headers = {'X-Api-Key': '9bDufSS8AE2NAOT2R5YtR9vutyukJlZh6ZQyhTnQ'}
response = requests.request("POST", url, headers=headers, data=payload,)
response_json=response.json()
response_json=pd.json_normalize(response_json['features'])



In [ ]:

response_json

,type,geometry.type,geometry.coordinates,properties.segment_id,properties.last_data_package,properties.timezone,properties.date,properties.period,properties.uptime,properties.heavy,properties.car,properties.bike,properties.pedestrian,properties.v85
0,Feature,MultiLineString,"[[[7.85632179465978, 47.9946964038859], [7.856...",9000002131,2022-07-21 08:07:23.256700+00:00,Europe/Berlin,2022-07-21 06:00:00+00:00,hourly,0.77,309.090909,1116.883117,592.207792,51.948052,35.0
1,Feature,MultiLineString,"[[[7.84317729466333, 47.9938840038842], [7.842...",9000002278,2022-07-21 08:05:36.323700+00:00,Europe/Berlin,2022-07-21 06:00:00+00:00,hourly,0.550833,30.86233,174.281392,176.096823,10.892587,15.0
2,Feature,MultiLineString,"[[[7.83662139466394, 47.9843122038866], [7.836...",9000002313,2022-07-21 08:07:24.882100+00:00,Europe/Berlin,2022-07-21 06:00:00+00:00,hourly,0.526389,271.662269,526.226913,484.432718,74.08971,29.5
3,Feature,MultiLineString,"[[[7.8597269946613, 48.0142050038795], [7.8604...",9000002438,2022-06-13 16:13:29.082700+00:00,Europe/Berlin,,,,,,,,
4,Feature,MultiLineString,"[[[7.85325729466223, 48.007344503881], [7.8532...",9000002543,2022-07-21 08:08:29.127100+00:00,Europe/Berlin,2022-07-21 06:00:00+00:00,hourly,0.712222,11.232449,87.051482,203.588144,32.293292,30.5
5,Feature,MultiLineString,"[[[7.81726769467111, 47.9985348038787], [7.816...",9000002571,2021-12-20 09:01:05.874900+00:00,Europe/Berlin,,,,,,,,
6,Feature,MultiLineString,"[[[7.81134179467293, 47.9999279038773], [7.811...",9000003492,2022-07-21 08:07:49.110400+00:00,Europe/Berlin,2022-07-21 06:00:00+00:00,hourly,0.0,0.0,0.0,0.0,0.0,
7,Feature,MultiLineString,"[[[7.843815694662, 47.9847759038875], [7.84338...",9000003844,2022-07-21 08:07:08.759100+00:00,Europe/Berlin,2022-07-21 06:00:00+00:00,hourly,0.535,71.028037,364.485981,259.813084,3.738318,11.5
8,Feature,MultiLineString,"[[[7.82270609466905, 47.994233003881], [7.8233...",9000003924,2022-07-21 08:07:46.641200+00:00,Europe/Berlin,2022-07-21 06:00:00+00:00,hourly,0.589167,84.865629,208.769448,169.731259,1.697313,25.5


In [ ]:
for result in response_json['features']:
    result = {result["geometry"]: result["properties"]}
result    

TypeError: unhashable type: 'dict'

In [ ]:
name_to_ids = {
    result['features'], result['type']
    for result in response_json
}

SyntaxError: invalid syntax (3321700694.py, line 3)